<a href="https://colab.research.google.com/github/Gajalakshmi993/Harvard_Project/blob/main/harvard_artifacts_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
import json
import pandas as pd
from pprint import pprint
API_key = "d2233ad8-60bc-4d3c-97c9-5580ad7b190e"
url = "https://api.harvardartmuseums.org/classification"

params = {
    "apikey": API_key,
    "size": 100
}

response = requests.get(url, params)
data = response.json()
for i in data['records']:
  obj_count = i['objectcount']
  if obj_count > 2500:
    print(i['name'], "    :     ", i['objectcount'])

Accessories (non-art)     :      2861
Photographs     :      85799
Drawings     :      33960
Prints     :      72252
Paintings     :      6863
Sculpture     :      6643
Coins     :      20270
Vessels     :      6223
Textile Arts     :      3446
Archival Material     :      15737
Fragments     :      5218
Manuscripts     :      5001
Seals     :      5899
Straus Materials     :      5128


In [17]:
pip install mysql-connector-python

In [18]:
!pip install streamlit streamlit_option_menu  # installing streamlit and streamlit_option_menu packages

In [31]:
%%writefile app.py
import requests
import pandas as pd
import streamlit as st
from streamlit_option_menu import option_menu
import mysql.connector

# ---------------- CONFIG ----------------
API_KEY = "d2233ad8-60bc-4d3c-97c9-5580ad7b190e"
BASE_URL = "https://api.harvardartmuseums.org/object"

connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    user="2ZXpEbV7H6oqcRk.root",
    password="ukBi5oJt2r1h9Udi",
    port=4000,
    database="artifacts"
)
cursor = connection.cursor()

# ---------------- TABLES ----------------
def create_tables():
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS metadata (
        id INT PRIMARY KEY,
        title TEXT,
        culture TEXT,
        period TEXT,
        century TEXT,
        medium TEXT,
        dimensions TEXT,
        description VARCHAR(100),
        department TEXT,
        classification VARCHAR(100),
        accessionyear INT,
        accessionmethod TEXT
    ) ENGINE=InnoDB;
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS media (
        objectid INT,
        imagecount INT,
        mediacount INT,
        colorcount INT,
        `rank` INT,
        datebegin INT,
        dateend INT,
        FOREIGN KEY (objectid) REFERENCES metadata(id)
        ON DELETE CASCADE
    ) ENGINE=InnoDB;
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS colors (
        objectid INT,
        color TEXT,
        spectrum TEXT,
        hue TEXT,
        percent FLOAT,
        css3 TEXT,
        FOREIGN KEY (objectid) REFERENCES metadata(id)
        ON DELETE CASCADE
    ) ENGINE=InnoDB;
    """)

    connection.commit()

create_tables()

# ---------------- API DATA ----------------
def harvard(api_key, classification):
    all_records = []
    for page in range(1, 26):
        params = {
            "apikey": api_key,
            "classification": classification,
            "size": 100,
            "page": page
        }
        response = requests.get(BASE_URL, params=params).json()
        all_records.extend(response.get("records", []))
    return all_records

# ---------------- PROCESS DATA ----------------
def artifact_details(records):
    metadata, media, colors = [], [], []

    for i in records:
        obj_id = i.get("id")

        metadata.append({
            "id": obj_id,
            "title": i.get("title"),
            "culture": i.get("culture"),
            "period": i.get("period"),
            "century": i.get("century"),
            "medium": i.get("medium"),
            "dimensions": i.get("dimensions"),
            "description": i.get("description"),
            "department": i.get("department"),
            "classification": i.get("classification"),
            "accessionyear": i.get("accessionyear"),
            "accessionmethod": i.get("accessionmethod")
        })

        media.append({
            "objectid": obj_id,
            "imagecount": i.get("imagecount"),
            "mediacount": i.get("mediacount"),
            "colorcount": i.get("colorcount"),
            "rank": i.get("rank"),
            "datebegin": i.get("datebegin"),
            "dateend": i.get("dateend")
        })

        for c in i.get("colors", []):
            colors.append({
                "objectid": obj_id,
                "color": c.get("color"),
                "spectrum": c.get("spectrum"),
                "hue": c.get("hue"),
                "percent": c.get("percent"),
                "css3": c.get("css3")
            })

    return metadata, media, colors

# ---------------- INSERT DATA ----------------
def insert_values(metadata, media, colors):
    cursor.executemany("""
        INSERT IGNORE INTO metadata VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, [(m["id"], m["title"], m["culture"], m["period"], m["century"],
           m["medium"], m["dimensions"], m["description"], m["department"],
           m["classification"], m["accessionyear"], m["accessionmethod"])
          for m in metadata])

    cursor.executemany("""
        INSERT INTO media VALUES (%s,%s,%s,%s,%s,%s,%s)
    """, [(m["objectid"], m["imagecount"], m["mediacount"], m["colorcount"],
           m["rank"], m["datebegin"], m["dateend"])
          for m in media])

    cursor.executemany("""
        INSERT INTO colors VALUES (%s,%s,%s,%s,%s,%s)
    """, [(c["objectid"], c["color"], c["spectrum"],
           c["hue"], c["percent"], c["css3"])
          for c in colors])

    connection.commit()

# ---------------- STREAMLIT UI ----------------
st.set_page_config(layout="wide")
st.title("🎨🏛️ Harvard Artifacts Collection")

classification = st.selectbox(
    "Select Classification",
    ["Coins", "Vessels", "Paintings"],
    index=None,
    placeholder="Choose classification"
)

menu = option_menu(
    None,
    ["Collect Data", "Migrate to SQL", "SQL Queries"],
    orientation="Vertical"
)

# -------- COLLECT --------
if menu == "Collect Data" and classification:
    records = harvard(API_KEY, classification)
    meta, med, col = artifact_details(records)

    c1, c2, c3 = st.columns(3)
    with c1: st.json(meta[:10])
    with c2: st.json(med[:10])
    with c3: st.json(col[:10])

# -------- MIGRATE --------
elif menu == "Migrate to SQL" and classification:
    if st.button("Insert into Database"):
        records = harvard(API_KEY, classification)
        meta, med, col = artifact_details(records)
        insert_values(meta, med, col)
        st.success("Data inserted successfully!")

        st.divider()

        # -------- SHOW METADATA --------
        st.subheader("📄 Metadata Table")
        cursor.execute("SELECT * FROM metadata")
        meta_rows = cursor.fetchall()
        meta_cols = [col[0] for col in cursor.description]
        df_meta = pd.DataFrame(meta_rows, columns=meta_cols)
        st.dataframe(df_meta, use_container_width=True)

        # -------- SHOW MEDIA --------
        st.subheader("🖼️ Media Table")
        cursor.execute("SELECT * FROM media")
        media_rows = cursor.fetchall()
        media_cols = [col[0] for col in cursor.description]
        df_media = pd.DataFrame(media_rows, columns=media_cols)
        st.dataframe(df_media, use_container_width=True)

        # -------- SHOW COLORS --------
        st.subheader("🎨 Colors Table")
        cursor.execute("SELECT * FROM colors")
        color_rows = cursor.fetchall()
        color_cols = [col[0] for col in cursor.description]
        df_colors = pd.DataFrame(color_rows, columns=color_cols)
        st.dataframe(df_colors, use_container_width=True)
# -------- QUERIES --------
elif menu == "SQL Queries":
    q = st.selectbox("Choose Queries", [
                  "1.List all artifacts from the 11th century belonging to Byzantine culture.",
                  "2.What are the unique cultures represented in the artifacts?",
                  "3.List all artifacts from the Archaic Period",
                  "4.List artifact titles ordered by accession year in descending order.",
                  "5.How many artifacts are there per department?",
                 "6.Which artifacts have more than 1 image?",
                 "7.What is the average rank of all artifacts?",
                 "8.Which artifacts have a higher colorcount than mediacount?",
                 "9.List all artifacts created between 1500 and 1600.",
                 "10.How many artifacts have no media files?",
                 "11.What are all the distinct hues used in the dataset?",
                 "12.What are the top 5 most used colors by frequency?",
                 "13.What is the average coverage percentage for each hue?",
                 "14.List all colors used for a given artifact ID.",
                 "15.What is the total number of color entries in the dataset?",
                 "16.List artifact titles and hues for all artifacts belonging to the Byzantine culture.",
                 "17.List each artifact title with its associated hues.",
                 "18.Get artifact titles, cultures, and media ranks where the period is not null.",
                 "19.Find artifact titles ranked in the top 10 that include the color hue 'Grey'.",
                 "20.How many artifacts exist per classification, and what is the average media count for each?",
                 "21.List all artifacts for which accession method is not recorded",
                 "22.Show objid and title where the  colorcount  b/w 5 and 8",
                 "23.Pick all spectrum, css3  for which hue is Green",
                 "24.Show the artifact titles when the accession year is 2002",
                 "25.what is the count of color in descending order when ccs3 is #2f4f4f"
    ], index=None, placeholder="Select a Query")

    if q == "1.List all artifacts from the 11th century belonging to Byzantine culture.":
        cursor.execute("""
            select * from metadata
            where culture = "Byzantine" and century = "11th century";
        """)
    elif q == "2.What are the unique cultures represented in the artifacts?":
        cursor.execute("""
          select distinct culture from metadata
           where culture is not null;
        """)

    elif q == "3.List all artifacts from the Archaic Period":
        cursor.execute("""
          select * from metadata
          where period like "%Archaic%";;
        """)
    elif q == "4.List artifact titles ordered by accession year in descending order.":
        cursor.execute("""
          select title from metadata
          order by accessionyear desc;
        """)
    elif q == "5.How many artifacts are there per department?":
        cursor.execute("""
         select department, count(*) as Artifact_count from metadata
         group by department;
        """)
    elif q == "6.Which artifacts have more than 1 image?":
        cursor.execute("""
          select objectid, imagecount from media
          where imagecount>1;
        """)
    elif q == "7.What is the average rank of all artifacts?":
        cursor.execute("""
          select avg(`rank`) as average_rank from media;
        """)
    elif q == "8.Which artifacts have a higher colorcount than mediacount?":
        cursor.execute("""
          select objectid, colorcount, mediacount from media
          where colorcount > mediacount;
        """)
    elif q == "9.List all artifacts created between 1500 and 1600.":
        cursor.execute("""
          select * from media
          where datebegin >= 1500  and dateend <= 1600;
        """)
    elif q == "10.How many artifacts have no media files?":
        cursor.execute("""
          select count(objectid) from media
          where mediacount is null or mediacount = " ";
        """)
    elif q == "11.What are all the distinct hues used in the dataset?":
        cursor.execute("""
          select distinct hue from colors;
        """)
    elif q == "12.What are the top 5 most used colors by frequency?":
        cursor.execute("""
          SELECT color, COUNT(*) AS frequency FROM colors
           GROUP BY color
           ORDER BY frequency DESC LIMIT 5
        """)
    elif q == "13.What is the average coverage percentage for each hue?":
        cursor.execute("""
          select distinct hue, round(avg(percent), 2) as avg_coverage_percentage from colors
           where percent is not null
           group by hue
           order by avg_coverage_percentage desc;
        """)
    elif q == "14.List all colors used for a given artifact ID.":
        cursor.execute("""
          select color from colors
          where objectid = 140930;
        """)
    elif q == "15.What is the total number of color entries in the dataset?":
        cursor.execute("""
          select count(*) as tot_no_colors from colors;
        """)
    elif q == "16.List artifact titles and hues for all artifacts belonging to the Byzantine culture.":
        cursor.execute("""
          select cm.title, cc.hue from metadata as cm
          join colors as cc on cm.id=cc.objectid
          where culture = "Byzantine";
        """)
    elif q == "17.List each artifact title with its associated hues.":
        cursor.execute("""
          select distinct cm.title, cc.hue from metadata as cm
          join colors as cc on cm.id=cc.objectid;
        """)
    elif q == "18.Get artifact titles, cultures, and media ranks where the period is not null.":
        cursor.execute("""
          select cm.title, cm.culture, m.`rank` from metadata as cm
          join media as m on cm.id=m.objectid
          where period is not null;
        """)
    elif q == "19.Find artifact titles ranked in the top 10 that include the color hue 'Grey'.":
        cursor.execute("""
          select cm.title, m.`rank`, cc.hue from metadata as cm
          join media as m on cm.id=m.objectid
          join colors as cc on cm.id=cc.objectid
          where hue like "_rey" limit 10;
        """)
    elif q == "20.How many artifacts exist per classification, and what is the average media count for each?":
        cursor.execute("""
          select  cm.classification, count(*) as artifact_count,  round(avg(m.mediacount), 2) as avg_media_count from metadata cm
          left join media as m  on m.objectid = cm.id
          group by cm.classification order by artifact_count desc;
        """)
    elif q == "21.List all artifacts for which accession method is not recorded":
        cursor.execute("""
          select * from metadata
          where accessionmethod like "Not %ecor%";
        """)
    elif q == "22.Show objid and title where the  colorcount  b/w 5 and 8":
        cursor.execute("""
          select m.objectid, cm.title from metadata as cm
          join media as m on m.objectid=cm.id
          where colorcount>5 and colorcount<8;
        """)
    elif q == "23.Pick all spectrum, css3  for which hue is Green":
        cursor.execute("""
          select spectrum, css3 from colors
           where hue = "Green";
        """)
    elif q == "24.Show the artifact titles when the accession year is 2002":
        cursor.execute("""
          select title from metadata where accessionyear = 2002;
        """)
    elif q == "25.what is the count of color in descending order when ccs3 is #2f4f4f":
        cursor.execute("""
          select color, count(color) as color_count from colors
          where css3 = "#2f4f4f"
          group by color
          order by color_count desc;
        """)
    else:
        cursor.execute("SELECT * FROM metadata")

    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    st.dataframe(df)


Overwriting app.py


In [13]:
# @title Setup code
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
import subprocess
subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

--2025-12-17 12:58:55--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-17 12:58:56--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-17T13%3A32%3A47Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-17

In [14]:
!streamlit run /content/app.py &>/content/logs.txt &

In [15]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://produce-desire-signal-ste.trycloudflare.com
